## Get runs from baselines


<Runs raymondl/tinystories-1m_play>


Processing runs:   0%|          | 0/105 [00:00<?, ?it/s]

Run e2e_bayesian_bayesian_seed-0_lpcoeff-0.05_logits-kl-1.0_lr-0.001_ratio-50.0_blocks.4.hook_resid_pre is not finished, skipping
Run local_bayesian_bayesian_seed-0_lpcoeff-9e-05_in-to-out-1.0_lr-0.001_ratio-50.0_blocks.4.hook_resid_pre is not finished, skipping


Processing runs: 100%|██████████| 105/105 [00:01<00:00, 59.36it/s]


,name,sparsity_coeff,L0,CELossIncrease,alive_dict_elements
41,ds_seed-0_lpcoeff-50.0_logits-kl-0.5_in-to-ori...,50.0,20.826889,0.146371,1914
45,ds_seed-0_lpcoeff-30.0_logits-kl-0.5_in-to-ori...,30.0,30.282940,0.101824,1923
49,ds_seed-0_lpcoeff-20.0_logits-kl-0.5_in-to-ori...,20.0,41.705284,0.075021,1974
53,ds_seed-0_lpcoeff-10.0_logits-kl-0.5_in-to-ori...,10.0,70.539283,0.042366,1984
57,ds_seed-0_lpcoeff-5.0_logits-kl-0.5_in-to-orig...,5.0,108.860445,0.021564,2013
61,ds_seed-0_lpcoeff-3.0_logits-kl-0.5_in-to-orig...,3.0,134.906909,0.014113,2024
67,ds_seed-0_lpcoeff-1.0_logits-kl-0.5_in-to-orig...,1.0,191.504038,0.003160,2093


Loaded pretrained model roneneldan/TinyStories-1M into HookedTransformer
Moving model to device:  cuda:0


ModuleDict(
  (blocks-4-hook_resid_pre): SAE(
    (encoder): Sequential(
      (0): Linear(in_features=64, out_features=3200, bias=True)
      (1): ReLU()
    )
    (decoder): Linear(in_features=3200, out_features=64, bias=True)
  )
)

In [29]:
model.raw_sae_positions

['blocks.4.hook_resid_pre']

In [65]:

import torch
import json
from collections.abc import Sequence
from functools import partial
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import wandb
import heapq
from e2e_sae import SAETransformer
from e2e_sae.log import logger
from e2e_sae.plotting import plot_facet, plot_per_layer_metric
from e2e_sae.scripts.analysis.plot_settings import (
    SIMILAR_CE_RUNS,
    SIMILAR_RUN_INFO,
    STYLE_MAP,
)
from e2e_sae.scripts.analysis.utils import create_run_df, get_df_gpt2
from e2e_sae.scripts.analysis.plot_performance import format_two_axes
device = torch.device('cuda:0')
api = wandb.Api(api_key='b8fa6d3104a0f99ee8a99f7c7659b893559f1097')
project = "raymondl/tinystories-1m_play"
runs = api.runs(project)
print(runs)
df = create_run_df(runs, per_layer_metrics=False, use_run_name=True, grad_norm=False)
df = df[df['name'].str.contains('blocks.4.hook_resid_pre')]
df = df[df['name'].str.contains('ds_seed')]
df[['name', 'sparsity_coeff', 'L0', 'CELossIncrease', 'alive_dict_elements']]
model = SAETransformer.from_wandb(f"raymondl/tinystories-1m_play/{df['id'].iloc[0]}").to('cuda:0')
model.saes.eval()
import math

from datasets import IterableDataset
from tqdm import tqdm

from e2e_sae.data import DatasetConfig, create_data_loader

config = runs[0].config
eval_config = DatasetConfig(**config['eval_data'])
eval_loader = create_data_loader(
    eval_config, batch_size=config['batch_size'], global_seed=config['seed']
)[0]
total_tokens = 0
TOP_K = 20
W = 64

cache_positions: list[str] | None = None
if config['loss']['in_to_orig'] is not None:
    assert set(config['loss']['in_to_orig']['hook_positions']).issubset(
        set(model.tlens_model.hook_dict.keys())
    ), "Some hook_positions in config.loss.in_to_orig.hook_positions are not in the model."
    # Don't add a cache position if there is already an SAE at that position which will cache
    # the inputs anyway
    cache_positions = [
        pos for pos in config.loss.in_to_orig.hook_positions if pos not in model.raw_sae_positions
    ]
if config.get('n_samples') is None:
    # If streaming (i.e. if the dataset is an IterableDataset), we don't know the length
    n_batches = None if isinstance(eval_loader.dataset, IterableDataset) else len(eval_loader)
else:
    n_batches = math.ceil(config['n_samples'] / config['batch_size'])

feature_counts  = None          # will be [n_features] tensor
feature_top     = None            # length = n_features; each is a min‑heap of (act, window_tokens, per_token_acts)

for batch_idx, batch in tqdm(enumerate(eval_loader), total=n_batches, desc="Eval Steps"):
    if n_batches is not None and batch_idx >= n_batches:
        break

    tokens = batch[eval_config.column_name].to(device)
    n_tokens = tokens.shape[0] * tokens.shape[1]
    total_tokens += n_tokens

    # Run through the SAE-augmented model
    with torch.no_grad():
        new_logits, new_acts = model.forward(
            tokens=tokens,
            sae_positions=model.raw_sae_positions,
            cache_positions=None,
        )
    coeffs = new_acts['blocks.4.hook_resid_pre'].c # Batch_size x seq_len x num_dictionary_elements
    B, S, N = coeffs.shape

    if feature_counts is None  and feature_top is None:
        feature_top = [ [] for _ in range(N) ]   # each is a min‑heap (max_act, toks, acts)
        feature_counts = torch.zeros(N, dtype=torch.long, device=device)
    
    n_w = math.ceil(S / W)

    #  --  reshape so axis‑1 indexes windows
    #  unfold returns a view of shape [B, n_w, W, N]
    coeffs_win = coeffs.unfold(dimension=1, size=W, step=W)   # [B, n_w, W, N]
    # tokens_win just for later token retrieval (CPU)
    tokens_win = tokens.unfold(1, W, W)                       # [B, n_w, W]

    # max activation of each feature *within* each window  ->  [B, n_w, N]
    max_per_win = coeffs_win.max(dim=2).values

    # Count how many tokens fire for each feature (activation>0)
    feature_counts += (coeffs > 0).sum(dim=(0,1)).to(torch.long)

    ############################################################################
    # 2.  For every feature, get its TOP_K windows (vectorised)
    ############################################################################
    # We process features in chunks to keep memory reasonable
    CHUNK = 256                       # tune for your GPU vRAM
    for start_f in range(0, N, CHUNK):
        f_slice = slice(start_f, start_f+CHUNK)
        # shape: [B*n_w, chunk]  (flatten batch+window so we can call topk once)
        flat = max_per_win[:,:,f_slice].reshape(-1, max_per_win.shape[-1])

        k_eff = min(TOP_K, flat.shape[0])
        if k_eff == 0:
            continue

        # torch.topk over rows = windows, per feature (dim=0)
        # top_vals, top_idx: [chunk, TOP_K]
        top_vals, top_idx = torch.topk(flat.t(), k=k_eff, largest=True)

        # turn flat index back into (b, w) pairs
        b_idx  = (top_idx // n_w).cpu()
        w_idx  = (top_idx %  n_w).cpu()
        top_vals = top_vals.cpu()

        for local_f, f_global in enumerate(range(start_f, min(start_f+CHUNK, N))):
            heap = feature_top[f_global]
            for rank in range(k_eff):
                act = top_vals[local_f, rank].item()
                if act == 0:             # no activation – ignore
                    break
                b   = b_idx [local_f, rank].item()
                w   = w_idx [local_f, rank].item()
                toks = tokens_win[b,w].cpu().tolist()
                acts = coeffs_win [b,w,:,f_global].cpu().tolist()

                if len(heap) < TOP_K:
                    heapq.heappush(heap,(act, toks, acts))
                elif act > heap[0][0]:   # replace worst
                    heapq.heapreplace(heap,(act, toks, acts))


<Runs raymondl/tinystories-1m_play>


Processing runs:   0%|          | 0/105 [00:00<?, ?it/s]

Run e2e_bayesian_bayesian_seed-0_lpcoeff-0.05_logits-kl-1.0_lr-0.001_ratio-50.0_blocks.4.hook_resid_pre is not finished, skipping
Run local_bayesian_bayesian_seed-0_lpcoeff-9e-05_in-to-out-1.0_lr-0.001_ratio-50.0_blocks.4.hook_resid_pre is not finished, skipping


Processing runs: 100%|██████████| 105/105 [00:01<00:00, 59.49it/s]


Loaded pretrained model roneneldan/TinyStories-1M into HookedTransformer
Moving model to device:  cuda:0


Eval Steps:   0%|          | 0/20000 [00:01<?, ?it/s]


IndexError: index 64 is out of bounds for dimension 0 with size 64

In [64]:
print(config)

{'lr': 0.001, 'loss': {'sparsity': {'l': -0.1, 'r': 1.1, 'beta': 0.3333333333333333, 'coeff': 0.001, 'p_norm': 1, 'epsilon': 1e-06}, 'logits_kl': None, 'out_to_in': {'coeff': 1}, 'in_to_orig': None, 'out_to_orig': None}, 'saes': {'retrain_saes': False, 'sae_positions': ['blocks.4.hook_resid_pre'], 'pretrained_sae_paths': None, 'dict_size_to_input_ratio': 50}, 'seed': 0, 'save_dir': '/home/raymond/.temp/e2e_sae/e2e_sae/scripts/train_tlens_saes/out', 'eval_data': {'seed': None, 'n_ctx': 512, 'split': 'validation', 'streaming': True, 'column_name': 'input_ids', 'dataset_name': 'apollo-research/roneneldan-TinyStories-tokenizer-gpt2', 'is_tokenized': True, 'tokenizer_name': 'gpt2'}, 'n_samples': 400000, 'batch_size': 20, 'train_data': {'seed': None, 'n_ctx': 512, 'split': 'train', 'streaming': True, 'column_name': 'input_ids', 'dataset_name': 'apollo-research/roneneldan-TinyStories-tokenizer-gpt2', 'is_tokenized': True, 'tokenizer_name': 'gpt2'}, 'lr_schedule': 'cosine', 'max_grad_norm': 1,

## ReLU vs No ReLU


In [23]:
api = wandb.Api(api_key='b8fa6d3104a0f99ee8a99f7c7659b893559f1097')
sweep_project = "raymondl/tinystories-1m-e2e-bayesian-beta-sweep"
runs = api.runs(sweep_project)
sweep_project_no_relu = "raymondl/tinystories-1m-e2e-bayesian-beta-sweep-no-relu"
runs_no_relu = api.runs(sweep_project_no_relu)
beta_values = set()
df = create_run_df(runs, per_layer_metrics=False, use_run_name=True, grad_norm=False)

df["grouping_type"] = "ReLU"
df_no_relu = create_run_df(runs_no_relu, per_layer_metrics=False, use_run_name=True, grad_norm=False)
# Filter df_no_relu to only include runs that are in df
df_no_relu = df_no_relu[df_no_relu["name"].isin(df["name"])]
df_no_relu["grouping_type"] = "No ReLU"
df = pd.concat([df, df_no_relu], axis=0)
plot_facet(
    df=df,
    xs=["CELossIncrease"],
    y="L0",
    facet_by="layer",
    facet_vals=[4],
    line_by="grouping_type",
    xlabels=["CE Loss Increase"],
    ylabel="L0",
    legend_title="Beta Value",
    axis_formatter=None,
    out_file="plots/bayesian_e2e_relu_vs_no_relu_layer_4.png",
    xlims=[{4: (None, None)}, {4: (None, None)}],
    ylim={4: (None, None)},
    styles=STYLE_MAP,
    plot_type='line',
    annotate_col="sparsity_coeff"
)

Processing runs: 100%|██████████| 130/130 [00:02<00:00, 50.00it/s]
2025-05-02 14:49:15 - INFO - Saved plot to plots/bayesian_e2e_relu_vs_no_relu_layer_4.png
2025-05-02 14:49:15 - INFO - Saved SVG plot to plots/bayesian_e2e_relu_vs_no_relu_layer_4.svg


## Learning Rates

In [27]:
api = wandb.Api(api_key='b8fa6d3104a0f99ee8a99f7c7659b893559f1097')
project = "raymondl/tinystories-1m-e2e-bayesian-beta-sweep-no-relu"
runs = api.runs(project)
df = create_run_df(runs, per_layer_metrics=False, use_run_name=True, grad_norm=False)
df = df[df['name'].str.contains('beta_0.5_bayesian_seed-0')]
df["grouping_type"] = df["name"].apply(lambda x: f"LR: {x.split('lr-')[-1].split('_')[0]}")
plot_facet(
    df=df,
    xs=["CELossIncrease"],
    y="L0",
    facet_by="layer",
    facet_vals=[4],
    line_by="grouping_type",
    xlabels=["CE Loss Increase"],
    ylabel="L0",
    legend_title="Beta Value",
    axis_formatter=None,
    out_file="plots/bayesian_e2e_learning_rates_layer_4.png",
    xlims=[{4: (None, None)}, {4: (None, None)}],
    ylim={4: (None, None)},
    styles=STYLE_MAP,
    plot_type='line',
    annotate_col="sparsity_coeff"
)

Processing runs: 100%|██████████| 130/130 [00:02<00:00, 50.49it/s]
2025-05-02 15:10:32 - INFO - Saved plot to plots/bayesian_e2e_learning_rates_layer_4.png
2025-05-02 15:10:32 - INFO - Saved SVG plot to plots/bayesian_e2e_learning_rates_layer_4.svg


## TinyStories-1M Comparisons with Baselines

In [19]:
api = wandb.Api(api_key='b8fa6d3104a0f99ee8a99f7c7659b893559f1097')
project = "raymondl/tinystories-1m_play"
runs = api.runs(project)
df = create_run_df(runs, per_layer_metrics=False, use_run_name=True, grad_norm=False)
df = df[df['name'].str.contains('blocks.4.hook_resid_pre')]
df = df[df['name'].str.contains('local_seed') | df['name'].str.contains('e2e_seed') | df['name'].str.contains('ds_seed')]
def assign_group(run_name: str):
    # if 'e2e_bayesian' in run_name or 'beta_0.5' in run_name:
    #     return 'e2e (bayesian)'
    # elif 'local_bayesian' in run_name:
    #     return 'local (bayesian)'
    if 'ds' in run_name:
        return 'ds'
    elif 'e2e' in run_name:
        return 'e2e'
    else:
        return 'local'
df["grouping_type"] = df["name"].apply(assign_group)

project = "tinystories-1m-e2e-bayesian-beta-annealing"
runs = api.runs(project)
df2 = create_run_df(runs, per_layer_metrics=False, use_run_name=True, grad_norm=False)
df2 = df2[df2['name'].str.contains('linear_annealing')]
def assign_group2(run_name: str):
    beta_value = f"{run_name.split('beta_')[-1].split('_')[0]}"
    return f"e2e (bayesian) {beta_value}"
df2["grouping_type"] = df2["name"].apply(assign_group2)



df = pd.concat([df, df2], axis=0)

# df = df[df['CELossIncrease'] < 1]
# df = df[df['L0'] < 100]


plot_facet(
    df=df,
    xs=["L0", "alive_dict_elements"],
    y="CELossIncrease",
    facet_by="layer",
    facet_vals=[4],
    line_by="grouping_type",
    xlabels=["L0", "Alive Dict Elements"],
    ylabel="CE Loss Increase",
    legend_title="SAE Type",
    axis_formatter=partial(format_two_axes, better_labels=True),
    out_file="plots/l0_vs_ce_loss_layer_4.png",
    xlims=[{4: (None, None)}, {4: (None, None)}],
    ylim={4: (.5, 0)},
    styles=STYLE_MAP,
    plot_type='line',
    # annotate_col="sparsity_coeff"
)

Processing runs:   0%|          | 0/105 [00:00<?, ?it/s]

Run e2e_bayesian_bayesian_seed-0_lpcoeff-0.05_logits-kl-1.0_lr-0.001_ratio-50.0_blocks.4.hook_resid_pre is not finished, skipping
Run local_bayesian_bayesian_seed-0_lpcoeff-9e-05_in-to-out-1.0_lr-0.001_ratio-50.0_blocks.4.hook_resid_pre is not finished, skipping


Processing runs: 100%|██████████| 51/51 [00:00<00:00, 643.18it/s]
/tmp/ipykernel_3183258/1399092007.py:31: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, df2], axis=0)
2025-05-03 18:24:07 - INFO - Saved plot to plots/l0_vs_ce_loss_layer_4.png
2025-05-03 18:24:07 - INFO - Saved SVG plot to plots/l0_vs_ce_loss_layer_4.svg


In [ ]:
api = wandb.Api(api_key='b8fa6d3104a0f99ee8a99f7c7659b893559f1097')
project = "raymondl/tinystories-1m_play"
runs = api.runs(project, filters={"tags": "local"})
df = create_run_df(runs, per_layer_metrics=False, use_run_name=True, grad_norm=False)

def assign_group(name):
    if "bayesian" in name:
        return "local (bayesian)"
    else:
        return "local"

df["grouping_type"] = df["name"].apply(assign_group)

plot_facet(
    df=df,
    xs=["CELossIncrease", "out_to_in"],
    y="L0",
    facet_by="layer",
    facet_vals=[4],
    line_by="grouping_type",
    xlabels=["CE Loss Increase", "Reconstruction MSE"],
    ylabel="L0",
    legend_title="SAE Type",
    axis_formatter=partial(format_two_axes, better_labels=True),
    out_file="plots/l0_vs_ce_loss_local_layer_4.png",
    xlims=[{4: (-0.5, 10)}, {4: (None, None)}],
    ylim={4: (None, None)},
    styles=STYLE_MAP,
    
    annotate_col="sparsity_coeff"
)


Processing runs:   0%|          | 0/61 [00:00<?, ?it/s]

Run local_bayesian_bayesian_seed-0_lpcoeff-9e-05_in-to-out-1.0_lr-0.001_ratio-50.0_blocks.4.hook_resid_pre is not finished, skipping


Processing runs: 100%|██████████| 61/61 [00:00<00:00, 182.75it/s]
2025-05-03 18:18:06 - INFO - Saved plot to plots/l0_vs_ce_loss_local_layer_4.png
2025-05-03 18:18:06 - INFO - Saved SVG plot to plots/l0_vs_ce_loss_local_layer_4.svg
